In [13]:
import robotic as ry
import numpy as np

from time import sleep





C = ry.Config()

maps = ["map1_simple_cross.g", "map2_tunnel.g", "map3_two_cave.g", "map4_one_cave.g", "map5_pioneer.g"]

C.addFile(f"maps/{maps[4]}")
C.view()
    




0